In [1]:
import tushare as ts

ts.set_token("cbe03b3c7468a5a82f6e3a215dca3da3468a2f861fa06ad8f2961828")
pro = ts.pro_api()


In [2]:
ts.get_hist_data('510500.SH')

本接口即将停止更新，请尽快使用Pro版接口：https://waditu.com/document/2


In [3]:
df = pro.daily(ts_code="206009",start_date='20210301')
df

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount


In [4]:
df = pro.fund_daily(ts_code='510500.SH', trade_date='20210307')
# df = pro.fund_daily(ts_code='510500.SH', start_date='20210101', end_date='20210303')
df

Exception: 抱歉，您没有访问该接口的权限，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。

In [1]:
import re
import requests
from datetime import datetime, timedelta, date
from lxml import etree
from pathlib import Path
from pprint import pprint
from typing import Generator, Tuple, Union

import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import numpy as np
import pandas as pd
import seaborn as sns
from dateutil.relativedelta import relativedelta
from jinja2 import Template

from etf_data import get_etf_dataframe, ETFData, ETFs, START_DATE, DATE_FMT
DATE_FMT = "%Y-%m-%d"

DateStr = Union[datetime, str]  # date could be represented as a string in the chosen format
Valuation = Tuple[np.datetime64, float]

In [2]:

# etf = ETFData("000311")
etf = ETFData("001051")
# df.drop(index=df.index, inplace=True)
# df.load()
# df.dtypes
# df.clear()
# df = pd.read_json("000311.json")

if not etf.df.empty:
    etf.df.drop(etf.df[etf.df["Date"] == etf.last_date].index, inplace=True)
    etf.df.drop(etf.df[etf.df["Date"] == etf.last_date].index, inplace=True)

print(etf.last_date)
print(etf.df.dtypes)
etf.update()
print(etf.last_date)
etf.df

# df.index
# df = pd.DataFrame()
# df.concat()

2021-03-17 00:00:00
Date                    datetime64[ns]
Unit Net Value                 float64
Cumulative Net Value           float64
Daily Growth                   float64
Purchase Status                 object
Selling Status                  object
Dividends                      float64
dtype: object
2021-03-19 00:00:00


,Date,Unit Net Value,Cumulative Net Value,Daily Growth,Purchase Status,Selling Status,Dividends
0,2015-03-17,1.0000,1.0000,NaN,封闭期,封闭期,NaN
1,2015-03-20,1.0000,1.0000,NaN,封闭期,封闭期,NaN
2,2015-03-27,1.0030,1.0030,NaN,封闭期,封闭期,NaN
3,2015-04-03,1.0130,1.0130,NaN,封闭期,封闭期,NaN
4,2015-04-10,1.0200,1.0200,NaN,封闭期,封闭期,NaN
...,...,...,...,...,...,...,...
1451,2021-03-15,1.2262,1.2262,-0.0175,开放申购,开放赎回,NaN
1452,2021-03-16,1.2368,1.2368,0.0086,开放申购,开放赎回,NaN
1453,2021-03-17,1.2339,1.2339,-0.0023,开放申购,开放赎回,NaN
1454,2021-03-18,1.2386,1.2386,0.0038,开放申购,开放赎回,NaN


## Semi-Conductors
* 008282	国泰CES半导体芯片行业	GTCESBDTXPHY	基金	
* 008888	华夏国证半导体芯片ETF	HXGZBDTXPETF	基金	
* 159801	广发国证半导体芯片ETF	GFGZBDTXPETF	基金	
* 512480	国联安中证半导体ETF	GLAZZBDTETF	基金	

In [3]:
import altair as alt

def draw_relative_graph(etfs:ETFs, ref_date:datetime):
    "Draw graph with altair of ETFs relative to a given date"
    etf_relative = [etf.get_relative_df(ref_date) for etf in etfs]
    etf_history_relative = pd.concat(etf_relative)


    data = alt.Chart(etf_history_relative).mark_line().encode(
        alt.X('Date:T'),
        alt.Y('Relative Net Value:Q', scale=alt.Scale(zero=False), axis=alt.Axis(format='%')),
        color='ETF',
        strokeDash='ETF',
    ).properties(width=1000, height=500)
    return data

    # start_points = pd.DataFrame([{"Date":np.datetime64(START_DATE) , "Ref": 1.0, "ETF": "N/A"}])
    # start_points = etf_history_relative[etf_history_relative["Date"] == START_DATE]
    # start_rule = alt.Chart(start_points).mark_rule().encode(
    #     alt.X('Date:T'),
    #     alt.Y('Relative Net Value:Q', scale=alt.Scale(zero=False), axis=alt.Axis(format='%')),
    #     color='ETF',
    # ).properties(width=1000, height=500)
    # alt.layer(data, start_rule).interactive()

In [4]:
CODES = ["206009", "001875", "005738", "004997", "161028"]
CODES = ["206009", "001875", "005738", "004997"]
# CODES = ["000311", "001051", "005880"]

# Semi-Conductors
# CODES = [
#  '008282',
#  '008888',
#  '159801',
#  '512480']

etfs = {code: ETFData(code).update() for code in CODES}

In [5]:
def get_chart(ref_date:datetime):
    """Get a chart based on given reference date"""
    data = draw_relative_graph(etfs.values(), ref_date)
    return alt.layer(data).properties(
        title=f"Relative Unit Net Value since {ref_date.strftime(DATE_FMT)}"
    )

def chart_to_svg(chart) -> str:
    """Get content of SVG corresponding to a chart"""
    svg_path = Path("history.svg")
    chart.save(str(svg_path))
    svg = svg_path.read_text()
    svg_path.unlink()
    return svg


today = datetime.today()
dates = {
    "1 week": today - relativedelta(weeks=1),
    "1 month": today - relativedelta(months=1),
    "3 months": today - relativedelta(months=3),
    "6 months": today - relativedelta(months=6),
    "1 year": today - relativedelta(years=1),
    "3 years": today - relativedelta(years=3),
}
chart_by_dates = {
    title:chart_to_svg(get_chart(date)) for title, date in dates.items()
} 

template = Template(Path("tmp_history.j2.html").read_text())
codes_as_str = ", ".join(etfs)
content = template.render(codes=codes_as_str, list_svg=chart_by_dates.items())
Path("etf_data.html").write_text(content)

ref_date = START_DATE
ref_date = datetime(2020, 3, 1)
get_chart(ref_date).interactive()

e:\AnsysDev\AFAC\afac_etf\etf_data.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Relative Net Value"] = df[VALUE_KEY] / ref_value - 1
e:\AnsysDev\AFAC\afac_etf\etf_data.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ETF"] = self.code


alt.LayerChart(...)

In [6]:
df = etfs["206009"].df
df = etfs["004997"].df
# df = etfs["001875"].df
# df[df["Date"] >= datetime(2019, 1, 1)]
df['Week Day'] = df['Date'].dt.day_name()
df["Day_of_Week"] = df.Date.dt.weekday
df["Week"] = df.apply(lambda x: x['Date'] - timedelta(days=x['Day_of_Week']), axis=1)
# print(df.dtypes)
df.style.format({'Daily Growth': "{:.3%}"})

df.pivot_table(index="Week Day", values="Daily Growth").style.format({'Daily Growth': "{:.3%}"})

,Daily Growth
Week Day,
Friday,0.078%
Monday,0.462%
Thursday,-0.085%
Tuesday,0.312%
Wednesday,-0.074%


In [7]:
import yfinance as yf

anss = yf.Ticker("ANSS")
# recom = anss.recommendations
anss.info
df = anss.history(period="max").reset_index()
# df.to_html("ANSS.html")
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1996-06-20,3.000000,3.125000,2.937500,2.968750,12638800,0,0.0
1,1996-06-21,3.031250,3.093750,3.000000,3.000000,1881200,0,0.0
2,1996-06-24,3.062500,3.218750,3.000000,3.218750,1606000,0,0.0
3,1996-06-25,3.218750,3.281250,3.156250,3.156250,1895600,0,0.0
4,1996-06-26,3.187500,3.187500,3.125000,3.156250,399200,0,0.0
...,...,...,...,...,...,...,...,...
6224,2021-03-15,314.769989,325.890015,314.769989,325.709991,496700,0,0.0
6225,2021-03-16,329.959991,333.239990,327.989990,330.779999,575100,0,0.0
6226,2021-03-17,326.369995,330.920013,321.359985,328.369995,477900,0,0.0
6227,2021-03-18,321.359985,327.440002,318.100006,321.489990,530000,0,0.0


In [8]:
import altair as alt


def draw_relative_graph_single(df:pd.DataFrame, ref_date:datetime, column_value:str = "Close"):
    "Draw graph with altair of single stock relative to a given date"
    df = df[df["Date"] >= ref_date]
    try:
        df.drop(["Volume", "Dividends", "Stock Splits"], axis=1, inplace=True)
    except:
        pass
    if not df.empty:
        ref_value = float(df.iloc[0][column_value])
        df["Relative Value"] = df[column_value] / ref_value - 1

    data = alt.Chart(df).mark_line().encode(
        alt.X('Date:T'),
        alt.Y('Relative Value:Q', scale=alt.Scale(zero=False), axis=alt.Axis(format='%')),
        # color='ETF',
        # strokeDash='ETF',
    ).properties(width=1000, height=500)
    return data

today = datetime.today()
draw_relative_graph_single(df, today - relativedelta(months=3)).interactive()

C:\Python37-32\lib\site-packages\pandas\core\frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
ipykernel_launcher:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


alt.Chart(...)

In [9]:
def get_chart_single(df:pd.DataFrame, ref_date:datetime):
    """Get a chart based on given reference date"""
    data = draw_relative_graph_single(df, ref_date)
    return alt.layer(data).properties(
        title=f"Relative Close Value since {ref_date.strftime(DATE_FMT)}"
    )


def chart_to_svg(chart) -> str:
    """Get content of SVG corresponding to a chart"""
    svg_path = Path("history.svg")
    chart.save(str(svg_path))
    svg = svg_path.read_text()
    svg_path.unlink()
    return svg


today = datetime.today()
dates = {
    "1 week": today - timedelta(weeks=1),
    "1 month": today - relativedelta(months=1),
    "3 months": today - relativedelta(months=3),
    "6 months": today - relativedelta(months=6),
    "1 year": today - relativedelta(years=1),
    "3 years": today - relativedelta(years=3),
    "5 years": today - relativedelta(years=5),
}
chart_by_dates = {
    title:chart_to_svg(get_chart_single(df, date)) for title, date in dates.items()
} 

template = Template(Path("tmp_history.j2.html").read_text())
codes_as_str = "Ansys Stock"
content = template.render(codes=codes_as_str, list_svg=chart_by_dates.items())
Path("ANSS_History.html").write_text(content)


ipykernel_launcher:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


248069

In [10]:
def get_grows_per_day(df: pd.DataFrame):
    df['Week Day'] = df['Date'].dt.day_name()
    df["Day_of_Week"] = df.Date.dt.weekday
    df["Week"] = df.apply(lambda x: x['Date'] - timedelta(days=x['Day_of_Week']), axis=1)
    # print(df.dtypes)
    if 'Daily Growth' not in df:
        df["Daily Growth"] = df.Close / df.shift().Close - 1.0
    df.style.format({'Daily Growth': "{:.3%}"})

    return df.pivot_table(index="Week Day", values="Daily Growth").style.format({'Daily Growth': "{:.3%}"})

get_grows_per_day(df)

,Daily Growth
Week Day,
Friday,0.098%
Monday,0.021%
Thursday,0.106%
Tuesday,0.155%
Wednesday,0.170%
